In [127]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

## $\text{Data Loading}$

In [40]:
# Running the Ipython notebook from the 'notebooks' directory
# The data path should point to '~/*/data' where the dataset is found
data_path = Path().absolute().parent.joinpath("data", "HistoricalPrices.csv")
spx_500 = pd.read_csv(data_path)

## $\text{Data Exploration}$

In [41]:
spx_500.shape

(2768, 5)

In [42]:
spx_500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2768 entries, 0 to 2767
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2768 non-null   object 
 1    Open   2768 non-null   float64
 2    High   2768 non-null   float64
 3    Low    2768 non-null   float64
 4    Close  2768 non-null   float64
dtypes: float64(4), object(1)
memory usage: 108.3+ KB


In [43]:
spx_500.duplicated().sum()

np.int64(0)

### $\text{Observation}$
$\text{The historical S\&P 500 index dataset has 2767 entries, with no missing data in any column.}$  <br>
$\text{The dataframe columns \textbf{\textit{Open}}, \textbf{\textit{High}},  \textbf{\textit{Low}}, and \textbf{\textit{Close}} are properly formated to the correct data type \textbf{float64}, but \textbf{\textit{Date}} is not in a \textit{datetime} format.}$

In [44]:
spx_500

,Date,Open,High,Low,Close
0,09/30/24,5726.52,5765.14,5703.53,5762.48
1,09/27/24,5755.36,5763.78,5727.34,5738.17
2,09/26/24,5762.22,5767.37,5721.01,5745.37
3,09/25/24,5733.65,5741.03,5712.06,5722.26
4,09/24/24,5727.66,5735.32,5698.99,5732.93
...,...,...,...,...,...
2763,10/07/13,1687.15,1687.15,1674.70,1676.12
2764,10/04/13,1678.79,1691.94,1677.33,1690.50
2765,10/03/13,1692.35,1692.35,1670.36,1678.66
2766,10/02/13,1691.90,1693.87,1680.34,1693.87


In [45]:
spx_500["Date"] = pd.to_datetime(spx_500["Date"], format="%m/%d/%y", yearfirst=True)

In [46]:
spx_500.sort_values(by="Date", ascending=True, ignore_index=True, inplace=True)

In [47]:
# CHECK: If the date is montonically increasing
spx_500.set_index("Date").index.is_monotonic_increasing

True

In [48]:
spx_500

,Date,Open,High,Low,Close
0,2013-10-01,1682.41,1696.55,1682.07,1695.00
1,2013-10-02,1691.90,1693.87,1680.34,1693.87
2,2013-10-03,1692.35,1692.35,1670.36,1678.66
3,2013-10-04,1678.79,1691.94,1677.33,1690.50
4,2013-10-07,1687.15,1687.15,1674.70,1676.12
...,...,...,...,...,...
2763,2024-09-24,5727.66,5735.32,5698.99,5732.93
2764,2024-09-25,5733.65,5741.03,5712.06,5722.26
2765,2024-09-26,5762.22,5767.37,5721.01,5745.37
2766,2024-09-27,5755.36,5763.78,5727.34,5738.17


## $\text{Preprocessing \& Feature Engineering}$


- $\text{50 and 200-day Moving Averages (MOV\_AVG 50/200D)}$
- $\text{14-day Relative Strength Index (RSI\_14D)}$
- $\text{Open and Closing Prices (PX\_OPEN/CLOSE)}$
- $\text{High and Low Prices (PX\_HIGH/LOW)}$
- $\text{Daily Price High-Low Difference (PX\_HIGH\_LOW\_DIFFERENCE)}$
- $\text{Daily Volume (PX\_VOLUME)}$
- $\text{30-day Volatility (VOLATILITY\_30D)}$
- $\text{Beta (BETA\_ADJ\_OVERRIDABLE)}$
  
$\text{Along with the historical SPX data, the following additional metrics are considered:}$

- $\textbf{SPX Ratios:}$
  + $\text{Price-to-Earnings Ratio (PE\_RATIO)}$
  + $\text{Price-to-Book Ratio (PX\_TO\_BOOK\_RATIO)}$
  + $\text{Price-to-Sales Ratio (PX\_TO\_SALES\_RATIO)}$
  + $\text{Earnings Yield (EARN\_YLD)}$
  
- $\textbf{Market Metrics:}$
  + $\text{Volatility Index (VIX)}$
  + $\text{10-Year Treasury Yield (USGG10YR)}$
  + $\text{NAPM Manufacturing PMI (NAPMPMI)}$
  + $\text{Consumer Confidence Index (CONCCONF)}$

### $\textbf{\textit{Market Metrics:}}$
### $\text{10-Year Treasury Yield (USGG10YR)}$

In [49]:
us_treasury_yeild = pd.read_csv("../data/us_treasury_yields_daily.csv")

In [50]:
us_treasury_yeild.head()

,date,US1M,US3M,US6M,US1Y,US2Y,US3Y,US5Y,US7Y,US10Y,US20Y,US30Y
0,1962-01-02,NaN,NaN,NaN,3.22,NaN,3.70,3.88,NaN,4.06,4.07,NaN
1,1962-01-03,NaN,NaN,NaN,3.24,NaN,3.70,3.87,NaN,4.03,4.07,NaN
2,1962-01-04,NaN,NaN,NaN,3.24,NaN,3.69,3.86,NaN,3.99,4.06,NaN
3,1962-01-05,NaN,NaN,NaN,3.26,NaN,3.71,3.89,NaN,4.02,4.07,NaN
4,1962-01-08,NaN,NaN,NaN,3.31,NaN,3.71,3.91,NaN,4.03,4.08,NaN


In [51]:
us_treasury_yeild_10Y = us_treasury_yeild[["date", "US10Y"]].copy()
del us_treasury_yeild

In [52]:
us_treasury_yeild_10Y["date"] = pd.to_datetime(us_treasury_yeild_10Y["date"], format="%Y-%m-%d", yearfirst=True)

In [53]:
us_treasury_yeild_10Y.rename(columns={"date": "Date", "US10Y": "USGG10YR_PX_LAST"}, inplace=True)

In [54]:
us_treasury_yeild_10Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16299 entries, 0 to 16298
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              16299 non-null  datetime64[ns]
 1   USGG10YR_PX_LAST  15604 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 254.8 KB


In [55]:
spx_500.columns

Index(['Date', ' Open', ' High', ' Low', ' Close'], dtype='object')

In [56]:
spx_500 = pd.merge(spx_500, us_treasury_yeild_10Y, on="Date", how="left")

### $\text{Volatility Index (VIX)}$

In [59]:
vix_daily = pd.read_csv("../data/vix_daily.csv")

In [60]:
vix_daily.head()

,date,open,high,low,close
0,1990-01-02,17.240000,17.240000,17.240000,17.240000
1,1990-01-03,18.190001,18.190001,18.190001,18.190001
2,1990-01-04,19.219999,19.219999,19.219999,19.219999
3,1990-01-05,20.110001,20.110001,20.110001,20.110001
4,1990-01-08,20.260000,20.260000,20.260000,20.260000


In [61]:
vix_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8685 entries, 0 to 8684
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    8685 non-null   object 
 1   open    8685 non-null   float64
 2   high    8685 non-null   float64
 3   low     8685 non-null   float64
 4   close   8685 non-null   float64
dtypes: float64(4), object(1)
memory usage: 339.4+ KB


In [62]:
vix_daily["date"] = pd.to_datetime(vix_daily["date"], yearfirst=True, format="%Y-%m-%d")

In [63]:
vix_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8685 entries, 0 to 8684
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    8685 non-null   datetime64[ns]
 1   open    8685 non-null   float64       
 2   high    8685 non-null   float64       
 3   low     8685 non-null   float64       
 4   close   8685 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 339.4 KB


In [64]:
vix_px_last = vix_daily[["date", "close"]].copy()
del vix_daily

In [65]:
vix_px_last.rename(columns={"date": "Date", "close": "VIX_PX_CLOSE"}, inplace=True)

In [78]:
vix_px_last.columns

Index(['Date', 'VIX_PX_LAST'], dtype='object')

In [79]:
spx_500 = pd.merge(spx_500, vix_px_last, how="left", on="Date")

In [76]:
spx_500.columns

Index(['Date', 'PX_OPEN', 'PX_HIGH', 'PX_LOW', 'PX_CLOSE', 'USGG10YR_PX_LAST',
       'PX_HIGH_LOW_DIFFERENCE', 'MOVING_AVG_50D', 'MOVING_AVG_200D'],
      dtype='object')

### $\text{NAPM Manufacturing PMI (NAPMPMI)}$

link: https://www.investopedia.com/terms/p/pmi.asp

> $\textit{
\textbf{What Is the Purchasing Managers' Index (PMI)?}}\\
\text{
The Purchasing Managers' Index (PMI) is an indicator of the prevailing direction of economic trends in the manufacturing and service}\\\text{sectors. The indicator is compiled and released monthly by the Institute for Supply Management (ISM), a nonprofit supply management}\\
\text{organization.}$

> $\text{It is a diffusion index that summarizes whether market conditions are expanding, staying the same, or contracting, as viewed by}\\\text{purchasing managers. The purpose of the PMI is to provide information about current and future business conditions to company}
\text{decision-makers, analysts, and investors.}$

> $\textbf{Formula and Calculation of the Purchasing Managers' Index (PMI)} $
$\text{The PMI is calculated as follows:}$
$$PMI = (P1 * 1) + (P2 * 0.5) + (P3 * 0)$$
$\textit{Where:}$
$\\
P1 = \text{percentage of answers reporting an improvement}  \\
P2 = \text{percentage of answers reporting no change} \\
P3 = \text{percentage of answers reporting a deterioration}$

In [123]:
pmi_index = pd.read_csv("../data/ISM-pmi-pm.csv")

In [128]:
pmi_index.head()

,period,PMI (ISM/pmi/pm)
0,2020-05,43.1
1,2020-06,52.2
2,2020-07,53.7
3,2020-08,55.6
4,2020-09,55.7


In [147]:
map_date_to_pmi = pmi_index.set_index("period").to_dict().get("PMI (ISM/pmi/pm)")
def map_date_pmi(x: pd.Series) -> float:
    year_month = x["Date"].strftime("%Y-%m")
    pmi = map_date_to_pmi.get(year_month, np.nan)
    return pmi


In [150]:
spx_500["NAPMPMI"] = spx_500.apply(map_date_pmi, axis=1)

### $\text{Consumer Confidence Index (CONCCONF)}$

In [158]:
consumer_confidence_index = pd.read_csv("../data/export-2025-02-11T17_37_10.215Z.csv", header=2)

In [159]:
consumer_confidence_index

,Category,OECD
0,2013-09-01 00:00:00,99.57681
1,2013-10-01 00:00:00,99.50415
2,2013-11-01 00:00:00,99.53159
3,2013-12-01 00:00:00,99.61710
4,2014-01-01 00:00:00,99.67163
...,...,...
131,2024-08-01 00:00:00,98.95267
132,2024-09-01 00:00:00,99.02384
133,2024-10-01 00:00:00,99.08997
134,2024-11-01 00:00:00,99.09577


In [170]:
consumer_confidence_index["Category"] = pd.to_datetime(consumer_confidence_index["Category"], format="%Y-%m-%d %H:%M:%S")
consumer_confidence_index["Category"] = consumer_confidence_index["Category"].dt.strftime("%Y-%m")

In [173]:
consumer_confidence_index

,Category,OECD
0,2013-09,99.57681
1,2013-10,99.50415
2,2013-11,99.53159
3,2013-12,99.61710
4,2014-01,99.67163
...,...,...
131,2024-08,98.95267
132,2024-09,99.02384
133,2024-10,99.08997
134,2024-11,99.09577


In [165]:
consumer_confidence_index.columns

Index(['Category', 'OECD'], dtype='object')

In [183]:
map_date_to_concconf = consumer_confidence_index.set_index("Category").to_dict().get("OECD")
def map_date_concconf(x: pd.Series) -> float:
    year_month = x["Date"].strftime("%Y-%m")
    oecd = map_date_to_concconf.get(year_month, np.nan)
    return oecd

In [185]:
spx_500["CONCCONF"] = spx_500.apply(map_date_concconf, axis=1)

In [66]:
# FIX: Remove leading whitespace found in column name `Open`, `Close`, `High`, and `Low`
spx_500.rename(columns=str.strip, inplace=True)

In [67]:
# FIX: Substitute current column names with those found in the research paper
col_names_to_sub =  {"Open": "PX_OPEN", "Close": "PX_CLOSE", "High": "PX_HIGH", "Low": "PX_LOW"}
spx_500.rename(columns=col_names_to_sub, inplace=True)

In [82]:
spx_500.columns

Index(['Date', 'PX_OPEN', 'PX_HIGH', 'PX_LOW', 'PX_CLOSE', 'USGG10YR_PX_LAST',
       'PX_HIGH_LOW_DIFFERENCE', 'MOVING_AVG_50D', 'MOVING_AVG_200D',
       'VIX_PX_LAST'],
      dtype='object')

In [69]:
# NEW COLUMN (PX_HIGH_LOW_DIFFERENCE): difference between the `PX_HIGH` and `PX_LOW`
spx_500["PX_HIGH_LOW_DIFFERENCE"] = spx_500["PX_HIGH"] - spx_500["PX_LOW"]

In [70]:
spx_500.head()

,Date,PX_OPEN,PX_HIGH,PX_LOW,PX_CLOSE,USGG10YR_PX_LAST,PX_HIGH_LOW_DIFFERENCE
0,2013-10-01,1682.41,1696.55,1682.07,1695.00,2.66,14.48
1,2013-10-02,1691.90,1693.87,1680.34,1693.87,2.63,13.53
2,2013-10-03,1692.35,1692.35,1670.36,1678.66,2.62,21.99
3,2013-10-04,1678.79,1691.94,1677.33,1690.50,2.66,14.61
4,2013-10-07,1687.15,1687.15,1674.70,1676.12,2.65,12.45


### $\text{Moving Average (50\&200-days)}$

$\textbf{formula:}$
$$\text{MOV\_AVG}=\frac{1}{N}\sum{_{i=0}^{N-1}}\textit{Close Price}_{i}$$

In [71]:
# NEW COLUMN (MOVING_AVG_50D): Moving average for 50 SPX market days
spx_500["MOVING_AVG_50D"] = spx_500["PX_CLOSE"].rolling(50).mean()

In [72]:
# NEW COLUMN (MOVING_AVG_200D): Moving average for 200 SPX market days
spx_500["MOVING_AVG_200D"] = spx_500["PX_CLOSE"].rolling(200).mean()

### $\text{Relative Strength Index (14-days)}$

$\textbf{formula:}$

$$\text{RSI}=100-\left(\frac{100}{1+RS}\right)$$

$\textbf{where}$

$$\text{RS}=\frac{\text{Average Gain Over 14 days}}{\text{Average Loss Over 14 days}}$$



In [73]:
def compute_rsi(df: pd.DataFrame=spx_500, window: int=14):
    delta = df["PX_CLOSE"].diff()
    avg_gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    avg_loss = (delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = avg_gain / avg_loss
    df[f"RSI_{window}D"] = 100 - (100 / (1 + RS))

In [74]:
spx_500

,Date,PX_OPEN,PX_HIGH,PX_LOW,PX_CLOSE,USGG10YR_PX_LAST,PX_HIGH_LOW_DIFFERENCE,MOVING_AVG_50D,MOVING_AVG_200D
0,2013-10-01,1682.41,1696.55,1682.07,1695.00,2.66,14.48,NaN,NaN
1,2013-10-02,1691.90,1693.87,1680.34,1693.87,2.63,13.53,NaN,NaN
2,2013-10-03,1692.35,1692.35,1670.36,1678.66,2.62,21.99,NaN,NaN
3,2013-10-04,1678.79,1691.94,1677.33,1690.50,2.66,14.61,NaN,NaN
4,2013-10-07,1687.15,1687.15,1674.70,1676.12,2.65,12.45,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2763,2024-09-24,5727.66,5735.32,5698.99,5732.93,NaN,36.33,5522.2118,5211.58160
2764,2024-09-25,5733.65,5741.03,5712.06,5722.26,NaN,28.97,5523.3130,5217.26495
2765,2024-09-26,5762.22,5767.37,5721.01,5745.37,NaN,46.36,5526.4550,5222.96995
2766,2024-09-27,5755.36,5763.78,5727.34,5738.17,NaN,36.44,5530.3266,5228.54860


### $\text{30-day Volatility (VOLATILITY 30D)}$

$\text{Stock market volatility is the rate at which a stock's price fluctuates over time. It's a measure of how risky an investment is. 
}$

$\textit{formula:}$
$$\text{Volatility}=\text{Standard Deviation of Log Returns over 30 days}\times\sqrt{252}$$ 

In [191]:
spx_500["VOLATILITY 30D"] = (np.log(spx_500["PX_CLOSE"] / spx_500["PX_CLOSE"].shift(1))).rolling(window=30).std() * np.sqrt(252)

In [192]:
spx_500.head()

,Date,PX_OPEN,PX_HIGH,PX_LOW,PX_CLOSE,USGG10YR_PX_LAST,PX_HIGH_LOW_DIFFERENCE,MOVING_AVG_50D,MOVING_AVG_200D,VIX_PX_LAST,NAPMPMI,CONCCONF,VOLATILITY 30D
0,2013-10-01,1682.41,1696.55,1682.07,1695.00,2.66,14.48,NaN,NaN,15.54,NaN,99.50415,NaN
1,2013-10-02,1691.90,1693.87,1680.34,1693.87,2.63,13.53,NaN,NaN,16.60,NaN,99.50415,NaN
2,2013-10-03,1692.35,1692.35,1670.36,1678.66,2.62,21.99,NaN,NaN,17.67,NaN,99.50415,NaN
3,2013-10-04,1678.79,1691.94,1677.33,1690.50,2.66,14.61,NaN,NaN,16.74,NaN,99.50415,NaN
4,2013-10-07,1687.15,1687.15,1674.70,1676.12,2.65,12.45,NaN,NaN,19.41,NaN,99.50415,NaN


In [193]:
spx_500.to_csv("../data/PreprocessedHistoricalPrices.csv", index=False)